<a href="https://colab.research.google.com/github/hanhanwu/Hanhan_COLAB_Experiemnts/blob/master/optimization_practice/gasoline_blending.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gasoline Blending

* [Problem Statement][1]
* Practice 2 dimensional variable

[1]:https://github.com/jckantor/ND-Pyomo-Cookbook/blob/main/notebooks/02.05-Gasoline-Blending.ipynb

In [1]:
import pandas as pd

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

import pyomo.environ as pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 1.3 MB/s eta 0:00:00
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 128097 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.11.4+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.108.6+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-2_amd64.deb ...
Unpacking coinor-libcgl1 (0.60.3+repack1-2) ...
Selecting previously unselecte

In [3]:
products = {
    'Regular' : {'price': 2.75, 'octane': 87, 'RVPmin': 0.0, 'RVPmax': 15.0, 'benzene': 1.1},
    'Premium' : {'price': 2.85, 'octane': 91, 'RVPmin': 0.0, 'RVPmax': 15.0, 'benzene': 1.1},
}

streams = {
    'Butane'       : {'RON': 93.0, 'MON': 92.0, 'RVP': 54.0, 'benzene': 0.00, 'cost': 0.85, 'avail': 30000},
    'LSR'          : {'RON': 78.0, 'MON': 76.0, 'RVP': 11.2, 'benzene': 0.73, 'cost': 2.05, 'avail': 35000},
    'Isomerate'    : {'RON': 83.0, 'MON': 81.1, 'RVP': 13.5, 'benzene': 0.00, 'cost': 2.20, 'avail': 0},
    'Reformate'    : {'RON':100.0, 'MON': 88.2, 'RVP':  3.2, 'benzene': 1.85, 'cost': 2.80, 'avail': 60000},
    'Reformate LB' : {'RON': 93.7, 'MON': 84.0, 'RVP':  2.8, 'benzene': 0.12, 'cost': 2.75, 'avail': 0},
    'FCC Naphtha'  : {'RON': 92.1, 'MON': 77.1, 'RVP':  1.4, 'benzene': 1.06, 'cost': 2.60, 'avail': 70000},
    'Alkylate'     : {'RON': 97.3, 'MON': 95.9, 'RVP':  4.6, 'benzene': 0.00, 'cost': 2.75, 'avail': 40000},
}
for s in streams.keys():
  streams[s]['octane'] = (streams[s]['RON'] + streams[s]['MON'])/2

In [13]:
model = pyomo.ConcreteModel()
P = products.keys()
S = streams.keys()
model.x = pyomo.Var(S, P, domain=pyomo.NonNegativeReals)

revenue = sum(sum(model.x[s, p] * products[p]['price'] for p in P) for s in S)
cost = sum(sum(model.x[s, p] * streams[s]['cost'] for s in S) for p in P)
model.profit = pyomo.Objective(expr = revenue - cost, sense=pyomo.maximize)

model.consts = pyomo.ConstraintList()
for s in S:
  model.consts.add(sum(model.x[s, p] for p in P) <= streams[s]['avail'])
model.consts.add(sum(model.x[s, p] * (streams[s]['octane'] - products[p]['octane']) for s in S for p in P) >= 0)
model.consts.add(sum(model.x[s, p] * (streams[s]['benzene'] - products[p]['benzene']) for s in S for p in P) <= 0)
model.consts.add(sum(model.x[s, p] * (streams[s]['RVP'] **1.25 - products[p]['RVPmin']**1.25) for s in S for p in P) >= 0)
model.consts.add(sum(model.x[s, p] * (streams[s]['RVP'] **1.25 - products[p]['RVPmax']**1.25) for s in S for p in P) <= 0)

solver = pyomo.SolverFactory('cbc')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 100425.0, 'Upper bound': 100425.0, 'Number of objectives': 1, 'Number of constraints': 12, 'Number of variables': 15, 'Number of nonzeros': 9, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 8}}, 'Error rc': 0, 'Time': 0.022835731506347656}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [19]:
total_profit = round(model.profit(), 2)
print(f'Optimized Total Profit: {total_profit} dollars')
total_vol = round(sum(model.x[s, p]() for s in S for p in P), 2)
print(f'Total Volume: {total_vol} gallon')
print(f'Profit per gallon: {round(total_profit*100/total_vol, 2)} cents per gallon')

Optimized Total Profit: 100425.0 dollars
Total Volume: 235000.0 gallon
Profit per gallon: 42.73 cents per gallon


In [23]:
df = pd.DataFrame()

for s in S:
  for p in P:
    df.loc[s, p] = model.x[s, p]() 
  df.loc[s,'total_avil'] = streams[s]['avail']

df['Total_vol'] = sum(df[p] for p in P)
df

,Regular,Premium,total_avil,Total_vol
Butane,30000.0,0.0,30000.0,30000.0
LSR,35000.0,0.0,35000.0,35000.0
Isomerate,0.0,0.0,0.0,0.0
Reformate,55750.0,4250.0,60000.0,60000.0
Reformate LB,0.0,0.0,0.0,0.0
FCC Naphtha,0.0,70000.0,70000.0,70000.0
Alkylate,0.0,40000.0,40000.0,40000.0
